In [7]:
# Model 1 - Baseline Linear Pricing
# This notebook implements the baseline linear pricing model where the parking price is updated in real-time based on occupancy.
# **formula used :**
# Price_(t+1) = Price_(t) + α * (Occupancy/Capacity)

# **key features:**
#- Starts from base price $10
#- α = 2 foe smooth adjusment
#- Price bounded between $5 and $20
#- Real-time simulation using pathway
#- Interactive Bokesh Visualizations.

In [2]:
# Importing Dataset
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset (1).csv


In [4]:
!pip install pathway bokeh

In [5]:
# Importing essential python libraries
import pandas as pd
import numpy as np
import pathway as pw
import json

In [6]:
# Reading the Dataset
df = pd.read_csv('/content/dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [8]:
# Preprocessing the dataset
# - combine data and time columns into a Timestamp
# - Encode categorical values for vehicleType and TrafficconditionNearby
# - ensure numerical values are integers(not float)

df = pd.read_csv('dataset.csv')
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], dayfirst = True)
vehicle_map = {"bike": 1, "car": 2, "truck": 3}
traffic_map = {"low": 1, "medium": 2, "high": 3}
df["VehicleTypeEncoded"] = df["VehicleType"].map(vehicle_map).fillna(0).astype(int)
df["TrafficEncoded"] = df["TrafficConditionNearby"].map(traffic_map).fillna(0).astype(int)
df = df.sort_values("Timestamp")
df.to_csv("preprocessed_dataset.csv",index = False)

In [9]:
# Define the input schema for pathway streaming
# Each column must match the structure of the dataset

class InputSchema(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str
    VehicleTypeEncoded: int
    TrafficEncoded: int

In [10]:
# Define baseline pricing logic (Model 1)
# Price is updated using: Price = Previous Price + α * (Occupancy / Capacity)
# Base price is $10, α = 2, price is clamped between $5 and $20

previous_price = {}

def pricing_logic(row):
  lot_id = row["SystemCodeNumber"]
  occupancy = row["Occupancy"]
  capacity = row["Capacity"]

  base_price = 10
  price_prev = previous_price.get(lot_id, base_price)

  alpha = 2
  price_new = price_prev + alpha * (occupancy / capacity)

  price_new = max(5, min(price_new, 20))
  previous_price[lot_id] = price_new

  return round(price_new, 2)

In [11]:
# Create and run Pathway pipeline using static mode
# - Simulates reading from a file as a data stream
# - Applies pricing logic using Pathway's UDF
# - Writes output to JSONLines file

@pw.udf
def calculate_price(lot_id, occupancy, capacity):
    row = {
        "SystemCodeNumber": lot_id,
        "Occupancy": occupancy,
        "Capacity": capacity
    }
    return pricing_logic(row)

input_table = pw.io.csv.read(
    "preprocessed_dataset.csv",
    schema=InputSchema,
    mode="static",
    autocommit_duration_ms=100
)

output_table = input_table.select(
    SystemCodeNumber=input_table.SystemCodeNumber,
    Timestamp=input_table.Timestamp,
    price=calculate_price(
        input_table.SystemCodeNumber,
        input_table.Occupancy,
        input_table.Capacity
    )
)

pw.io.jsonlines.write(output_table, "output.jsonlines")

pw.run()

Output()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [12]:
# Reading output data generated by pathway
# This includes final price per timestamp per lot

import json
import pandas as pd

output_data = []
with open("output.jsonlines", "r") as f:
  for line in f:
    output_data.append(json.loads(line))

output_df = pd.DataFrame(output_data)
output_df["Timestamp"] = pd.to_datetime(output_df['Timestamp'])

output_df.head()

,SystemCodeNumber,Timestamp,price,diff,time
0,BHMEURBRD01,2016-10-07 13:30:00,11.54,1,1751715630822
1,Shopping,2016-12-10 07:59:00,10.33,1,1751715630822
2,Others-CCCPS202,2016-12-06 14:29:00,10.88,1,1751715630822
3,Others-CCCPS202,2016-11-05 08:00:00,11.13,1,1751715630822
4,Others-CCCPS135a,2016-12-08 11:59:00,11.78,1,1751715630822


In [13]:
# Plot dynamic price trends using Bokeh
# Shows how the price evolved for 3 parking lots over time

from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Category10
from bokeh.models import HoverTool

output_notebook()


lots_to_plot = output_df['SystemCodeNumber'].unique()[:3]
colors = Category10[10]


p = figure(title="Price Over Time for Parking Lots (Model 1)",
           x_axis_type='datetime', width=900, height=400)


for i, lot in enumerate(lots_to_plot):
    lot_df = output_df[output_df['SystemCodeNumber'] == lot]
    p.line(
        x=lot_df['Timestamp'],
        y=lot_df['price'],
        line_width=2,
        color=colors[i % len(colors)],
        legend_label=f"{lot}"
    )


p.add_tools(HoverTool(
    tooltips=[("Time", "@x{%F %T}"), ("Price", "@y")],
    formatters={"@x": "datetime"},
    mode="vline"
))

p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"
p.legend.location = "top_left"
p.legend.click_policy = "hide"

show(p)

In [15]:
# Processed datasets in this model
preprocessed_dataset = pd.read_csv("/content/preprocessed_dataset.csv")
output_jsonlines = pd.read_csv("/content/output.jsonlines")